In [18]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

path = r'C:\Users\User\Instacart Basket Analysis'
df_customers=pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

## Wrangling, data quality and consistency checks

In [26]:
df_customers.shape

(206209, 10)

In [27]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### I will change the column name from Surnam to Last Name

In [28]:
df_customers.rename(columns = {'Surnam' : 'Last Name'}, inplace = True) 

In [29]:
df_customers.head()

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### I found that in tasks 4.9 and 4.10 columns  First Name, Last Name, Gender, and date_joined are not used. 
#### So, I deleted them.

In [30]:
df_customers = df_customers.drop(columns = ['First Name', 'Last Name', 'Gender', 'date_joined']) 

In [31]:
df_customers.head()

,user_id,STATE,Age,n_dependants,fam_status,income
0,26711,Missouri,48,3,married,165665
1,33890,New Mexico,36,0,single,59285
2,65803,Idaho,35,2,married,99568
3,125935,Iowa,40,0,single,42049
4,130797,Maryland,26,1,married,40374


### Checking for outliers

In [32]:
df_customers.describe() 

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### There are no outliers

#### 8.	Test for mixed data:

In [33]:
for col in df_customers.columns.tolist():
  wrd = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[wrd]) > 0:
    print (col)


#### No mixed data

#### Test for missing data:

In [34]:
df_customers.isnull().sum() 

user_id         0
STATE           0
Age             0
n_dependants    0
fam_status      0
income          0
dtype: int64

#### No missing data

#### Test for duplicates

In [35]:
df_dups = df_customers[df_customers.duplicated()] 

In [36]:
df_dups

,user_id,STATE,Age,n_dependants,fam_status,income


#### No duplicates

In [37]:
df_customers.shape

(206209, 6)

## Merge dataframes

In [19]:
# read dataframe from .pkl
df_4_9_ords_prods_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4_9_orders_products_merged.pkl'))

In [39]:
df_4_9_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,days_since_prior_order_Flag,add_to_cart_order,reordered,max_order,loyalty_flag,average_price,spender_flag,median_freq,freq_Flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,5,New customer,7.930208,Low spender,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,3,New customer,4.972414,Low spender,9.0,Frequent customer


In [38]:
df_customers.head()

,user_id,STATE,Age,n_dependants,fam_status,income
0,26711,Missouri,48,3,married,165665
1,33890,New Mexico,36,0,single,59285
2,65803,Idaho,35,2,married,99568
3,125935,Iowa,40,0,single,42049
4,130797,Maryland,26,1,married,40374


In [43]:
df_4_9_ords_prods_merge.dtypes['user_id'] 

dtype('int64')

In [44]:
df_customers.dtypes['user_id'] 

dtype('int64')

#### 'user_id' type is the same in both dataframes

In [40]:
# Only common column for both dataframes is 'user_id'. 
df_orders_products_all = df_customers.merge(df_4_9_ords_prods_merge, on = 'user_id', indicator = True)

In [41]:
df_orders_products_all.shape

(32404859, 26)

In [42]:
df_orders_products_all.head()

,user_id,STATE,Age,n_dependants,fam_status,income,product_id,product_name,aisle_id,department_id,...,days_since_prior_order_Flag,add_to_cart_order,reordered,max_order,loyalty_flag,average_price,spender_flag,median_freq,freq_Flag,_merge
0,26711,Missouri,48,3,married,165665,196,Soda,77,7,...,False,2,0,8,New customer,7.988889,Low spender,19.0,Regular customer,both
1,26711,Missouri,48,3,married,165665,196,Soda,77,7,...,False,1,1,8,New customer,7.988889,Low spender,19.0,Regular customer,both
2,26711,Missouri,48,3,married,165665,196,Soda,77,7,...,False,2,1,8,New customer,7.988889,Low spender,19.0,Regular customer,both
3,26711,Missouri,48,3,married,165665,6184,Clementines,32,4,...,True,1,0,8,New customer,7.988889,Low spender,19.0,Regular customer,both
4,26711,Missouri,48,3,married,165665,6184,Clementines,32,4,...,False,2,1,8,New customer,7.988889,Low spender,19.0,Regular customer,both


In [45]:
# Export as .pkl
df_orders_products_all.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_all.pkl'))